In [43]:
# -*- coding: utf-8 -*-

"""
根据shp文件中的矩形框裁剪tif影像
~~~~~~~~~~~~~~~~
code by wHy
Aerospace Information Research Institute, Chinese Academy of Sciences
wanghaoyu191@mails.ucas.ac.cn
"""

'\n根据shp文件中的矩形框裁剪tif影像\n~~~~~~~~~~~~~~~~\ncode by wHy\nAerospace Information Research Institute, Chinese Academy of Sciences\nwanghaoyu191@mails.ucas.ac.cn\n'

In [44]:
import geopandas as gpd 
import rasterio # rasterio现在必须在python>3.8的版本上才能正确用pip安装
from rasterio.mask import mask
import fnmatch
import os

In [45]:
shp_file_path = r'E:\project_UAV_GF2_2\0-clip_boundary\clip_boundary.shp'
tif_img_path = r'E:\image_UAV_REJECT'
output_path = r'E:\project_UAV_GF2_2\1-clip_img_UAV'

shp_path, shp__name = os.path.split(shp_file_path)

if not os.path.exists(output_path):
    os.mkdir(output_path)

listpic = fnmatch.filter(os.listdir(tif_img_path), '*.tif')

# 读取矩形信息的shp文件
rectangles = gpd.read_file(shp_file_path)

# 过滤的图像文件列表
GF2_IMG_list = ['GF2_PMS1_E84.4_N41.3_20240621_L1A13926016001-MSS1_MS_TOA_fuse.img', 'GF2_PMS1_E84.3_N41.1_20240621_L1A13926017001-MSS1_MS_TOA_fuse.img']
UAV_IMG_list = ['20240511_塔河-17.dat', '20240511_塔河-18.dat', '20240511_塔河-19.dat', '20240511_塔河-20.dat', '20240511-塔河-21.dat', '20240511_塔河-51.dat']


for idx, rectangle in rectangles.iterrows():
    # 获取矩形的几何信息
    geometry = rectangle.geometry
    
    # 打开大的TIF影像文件
    for img in listpic:

        # if img not in GF2_IMG_list: # 图像过滤
        #     continue

        img_full_path = tif_img_path + '/' + img

        with rasterio.open(img_full_path) as src:
            image_bounds = src.bounds  # 影像的边界

            # 获取矩形的边界并与影像的边界进行检查
            geom_bounds = geometry.bounds
            if not (geom_bounds[2] > image_bounds.left and  # geom maxX > img minX
                    geom_bounds[0] < image_bounds.right and  # geom minX < img maxX
                    geom_bounds[3] > image_bounds.bottom and  # geom maxY > img minY
                    geom_bounds[1] < image_bounds.top):       # geom minY < img maxY
                print(f"Geometry {idx} does not intersect with the image.")
                continue            

            # 将矩形的几何信息转换为GeoJSON格式
            geojson_geom = [geometry.__geo_interface__]

            # 使用rasterio的mask函数裁剪影像
            out_image, out_transform = mask(src, geojson_geom, crop=True)

            # 通过原始影像的transform属性创建新的裁剪后影像
            out_meta = src.meta.copy()
            out_meta.update({"driver": "GTiff",
                            "height": out_image.shape[1],
                            "width": out_image.shape[2],
                            "transform": out_transform,
                            "dtype":src.meta['dtype']}) # 注意与原始影像位数对齐

            # 保存裁剪后影像
            # output_filename =img[:-4]+ '_' + shp__name[:-4] + f'_output_{idx}.tif'
            output_filename =f'{idx}.tif'
            output_full_path = output_path + '/' + output_filename
            with rasterio.open(output_full_path, "w", **out_meta) as dest:
                dest.write(out_image)

Geometry 0 does not intersect with the image.
Geometry 0 does not intersect with the image.
Geometry 0 does not intersect with the image.
Geometry 1 does not intersect with the image.
Geometry 1 does not intersect with the image.
Geometry 1 does not intersect with the image.
Geometry 2 does not intersect with the image.
Geometry 2 does not intersect with the image.
Geometry 2 does not intersect with the image.
Geometry 3 does not intersect with the image.
Geometry 3 does not intersect with the image.
Geometry 3 does not intersect with the image.
Geometry 4 does not intersect with the image.
Geometry 4 does not intersect with the image.
Geometry 4 does not intersect with the image.
Geometry 5 does not intersect with the image.
Geometry 5 does not intersect with the image.
Geometry 5 does not intersect with the image.
Geometry 6 does not intersect with the image.
Geometry 6 does not intersect with the image.
Geometry 6 does not intersect with the image.
Geometry 7 does not intersect with